## Pre-schedule trips using Ridehail API

Currently, the Ridehail API supports on demand requests. In this tutorial, we will use this feature to cater to pre-scheduled trips. Here is what we are going to do:

1. Create a fleet and add a vehicle to the fleet.
2. Update the state (position and readiness) of the vehicle and set it to accept rides.
3. Create a pre-scheduled trip request.
4. Request another trip to simulate an on-demand request. A vehicle will be assigned to this trip only if it able to complete the trip before the pre-scheduled trip starts.
5. Complete all the steps of the on-demand trip.
6. Complete all the steps of the pre-scheduled trip.
7. Clean up vehicle and the fleet.

In [ ]:
import requests
import json
import uuid
from datetime import datetime,timedelta

## API Key

To run this example, you'll need a rideOS API key. You can sign up for one [here](https://app.rideos.ai/login#lockScreen=signUp) and view it on your [profile page](https://app.rideos.ai/profile), then assign it to the `API_KEY` variable below:


In [ ]:
# IMPORTANT: replace "YOUR_RIDEOS_API_KEY" with your actual rideOS API key
API_KEY = "YOUR_RIDEOS_API_KEY"

AUTHORIZATION_HEADER = {"X-Api-Key": API_KEY}

## Creating UUIDs

Creating fleet, vehicle and rider uuids using uuid4.

In [ ]:
fleet_id = str(uuid.uuid4())
vehicle_id = str(uuid.uuid4())
pre_scheduled_rider_id = str(uuid.uuid4())
on_demand_rider_id = str(uuid.uuid4())
pre_scheduled_trip_id = str(uuid.uuid4())
on_demand_trip_id = str(uuid.uuid4())

print("Fleet id: " + fleet_id)
print("Vehicle id: " + vehicle_id)
print("Pre-scheduled Rider id: " + pre_scheduled_rider_id)
print("On-demand Rider id: " + on_demand_rider_id)
print("Pre-scheduled trip id: " + pre_scheduled_trip_id)
print("On-demand trip id: " + on_demand_trip_id)

## Step 1:  Create a fleet

Creating a fleet.

In [ ]:
CREATE_FLEET_URL = "https://api.rideos.ai/ride-hail-operations/v1/CreateFleet"

create_fleet_request = {
    "id": fleet_id,
    "info": {
        "display_name": "Sample Application Fleet"
    }
}

response = requests.post(
    CREATE_FLEET_URL,
    headers=AUTHORIZATION_HEADER,
    json=create_fleet_request
)
response.raise_for_status()

create_fleet_response = response.json()

print("Got successful create fleet response:")
print(json.dumps(create_fleet_response, indent=4))


### Create a Vehicle

Adding a vehicle to a fleet using ride-hail-driver. Each vehicle can only belong to one fleet.

In [ ]:
CREATE_VEHICLE_URL = "https://api.rideos.ai/ride-hail-driver/v1/CreateVehicle"

create_vehicle_request = {
    "id": vehicle_id,
    "fleet_id": fleet_id,
    "definition": {
        "rider_capacity": 4
    },
    "info": {
        "driver_info": {
            "contact_info": {
                "name": "sample app driver1",
                "phone_number": 987654321
            }
        },
        "properties": {
            "make": "Toyota",
            "model": "Camry"
        }
    }
}
    
response = requests.post(
    CREATE_VEHICLE_URL,
    headers=AUTHORIZATION_HEADER,
    json=create_vehicle_request
)
response.raise_for_status()

create_vehicle_response = response.json()

print("Got successful create vehicle response:")
print(json.dumps(create_vehicle_response, indent=4))

## Step 2: Update vehicle state and position

Setting the vehicle's current position by updating the vehicle state.

In [ ]:
UPDATE_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/UpdateVehicleState"

update_vehicle_state_request = {
    "id": vehicle_id,
    "update_position": {
        "updated_position": { # San Francisco
            "latitude": 37.788897, 
            "longitude": -122.389669
        },
        "updated_heading": 0.0
    }
}
    
response = requests.post(
    UPDATE_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=update_vehicle_state_request
)
response.raise_for_status()

update_vehicle_state_response = response.json()

print("Got successful update vehicle state response:")
print(json.dumps(update_vehicle_state_response, indent=4))

### Update vehicle to accept rides

Marking the vehicle to accept rides by updating the vehicle state.

In [ ]:
UPDATE_VEHICLE_STATE_URL = "https://api.rideos.ai/ride-hail-driver/v1/UpdateVehicleState"

update_vehicle_state_request = {
    "id": vehicle_id,
    "set_to_accept_rides": {}
}
    
response = requests.post(
    UPDATE_VEHICLE_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=update_vehicle_state_request
)
response.raise_for_status()

update_vehicle_state_response = response.json()

print("Got successful update vehicle state response:")
print(json.dumps(update_vehicle_state_response, indent=4))

## Step 3: Create a prescheduled trip request (trip-1)

Create a trip request for a later time. We are going to create a trip with a pick up time window. In this example, the rider must be picked up at 30 minutes from now and no later than 35 minutes from now. 

In [ ]:
REQUEST_TRIP_URL = "https://api.rideos.ai/ride-hail-rider/v1/RequestTripRC"

request_trip_request = {
   "id": pre_scheduled_trip_id,
   "rider_id": pre_scheduled_rider_id,
   "fleet_id": fleet_id,
   "definition":{
      "pickup_dropoff":{
         "pickup":{
            "position":{
               "latitude":37.794682, 
               "longitude":-122.411480
            }
         },
         "dropoff":{
            "position":{
               "latitude": 37.802372,
               "longitude": -122.418729
            }
         },
         "rider_count":2
      }
   },
   "info":{
      "rider_info":{
         "contact_info":{
            "name":"sample app rider1",
            "phone_number":123456789
         }
      }
   },
   "dispatchParameters":{
      "requiredPickupTimeWindow":{
          "earliestTime": (datetime.utcnow()+timedelta(minutes=30)).strftime("%Y-%m-%dT%H:%M:%S%ZZ"),
          "latestTime":  (datetime.utcnow()+timedelta(minutes=35)).strftime("%Y-%m-%dT%H:%M:%S%ZZ")
      }
   }
}

response = requests.post(
    REQUEST_TRIP_URL,
    headers=AUTHORIZATION_HEADER,
    json=request_trip_request
)
response.raise_for_status()

request_trip_response = response.json()

print("Got successful request trip response:")
print(json.dumps(request_trip_response, indent=4))

### Get the current status of the trip 
Check the state of the rider's trip which should be `DrivingToPickup`. We will also get the step IDs of the next steps  required to succesfully complete the trip i.e drive to pickup location, pick up the rider, drive to destination, and drop off the rider.

Something to note: if GetTripStateRC is called really quickly after RequestTrip, we might not have a vehicle assigned yet. In that situation, wait a couple seconds and re-run the cell. 

In [ ]:
GET_TRIP_STATE_URL = "https://api.rideos.ai/ride-hail-rider/v1/GetTripStateRC"

get_trip_state_request = {
    "id": pre_scheduled_trip_id
}

response = requests.post(
    GET_TRIP_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_trip_state_request
)
response.raise_for_status()

get_trip_state_response = response.json()

pre_scheduled_step_ids = []
for i in range(4):
    step_id = get_trip_state_response['state']['drivingToPickup']['assignedVehicle']['planThroughTripEnd']['step'][i]['id']
    dict = {"stepId": step_id}
    pre_scheduled_step_ids.append(dict)
    
print("Got successful get trip state response:")
print(json.dumps(get_trip_state_response, indent=4))

## Step 4: Request another trip (trip-2)

Request another trip to simulate an on-demand request received in real time. In such a trip, we will set the drop off time as 30 mins from now. The vehicle will be assigned to this trip if the trip can be completed before the pre-schedule trip begins. The current status of the trip can be checked by calling the `GetTripStateRC` endpoint.

In [ ]:
REQUEST_TRIP_URL = "https://api.rideos.ai/ride-hail-rider/v1/RequestTripRC"

request_trip_request = {
   "id": on_demand_trip_id,
   "rider_id": on_demand_rider_id,
   "fleet_id": fleet_id,
   "definition":{
      "pickup_dropoff":{
         "pickup":{
            "position":{
               "latitude":37.790329,
               "longitude":-122.392231
            }
         },
         "dropoff":{
            "position":{
               "latitude":37.795107,
               "longitude":-122.393426
            }
         },
         "rider_count":2
      }
   },
   "info":{
      "rider_info":{
         "contact_info":{
            "name":"sample app rider2",
            "phone_number":123456789
         }
      }
   },
   "dispatchParameters":{
      "requiredDropoffTimeWindow":{
         "latestTime": (datetime.utcnow()+timedelta(minutes=30)).strftime("%Y-%m-%dT%H:%M:%S%ZZ")
      }
   }
}

response = requests.post(
    REQUEST_TRIP_URL,
    headers=AUTHORIZATION_HEADER,
    json=request_trip_request
)
response.raise_for_status()

request_trip_response = response.json()

print("Got successful request trip response:")
print(json.dumps(request_trip_response, indent=4))

### Get the current status of the trip
Check the state of the rider's trip which should be `DrivingToPickup`. We will also get the step IDs of the next steps  required to succesfully complete the trip i.e drive to pickup location, pick up the rider, drive to destination, and drop off the rider.

Something to note: if GetTripStateRC is called really quickly after RequestTrip, we might not have a vehicle assigned yet. In that situation, wait a couple seconds and re-run the cell. 

In [ ]:
GET_TRIP_STATE_URL = "https://api.rideos.ai/ride-hail-rider/v1/GetTripStateRC"

get_trip_state_request = {
    "id": on_demand_trip_id
}

response = requests.post(
    GET_TRIP_STATE_URL,
    headers=AUTHORIZATION_HEADER,
    json=get_trip_state_request
)
response.raise_for_status()

get_trip_state_response = response.json()

on_demand_step_ids = []
for i in range(4):
    step_id = get_trip_state_response['state']['drivingToPickup']['assignedVehicle']['planThroughTripEnd']['step'][i]['id']
    dict = {"stepId": step_id}
    on_demand_step_ids.append(dict)

print("Got successful get trip state response:")
print(json.dumps(get_trip_state_response, indent=4))

## Step 5: Complete on-demand trip (trip-2)
Complete all the steps of trip-2 from drive to pick up location to drop off rider.

#### Method to call `CompleteSteps` endpoint

In [ ]:
def complete_steps(trip_id, step_ids):
    COMPLETE_STEP_URL = "https://api.rideos.ai/ride-hail-driver/v1/CompleteSteps"

    complete_step_request = {
        "vehicle_id": vehicle_id,
        "trip_id": trip_id,
        "stepsToComplete": step_ids
    }

    response = requests.post(
        COMPLETE_STEP_URL,
        headers=AUTHORIZATION_HEADER,
        json=complete_step_request
    )
    response.raise_for_status()

    complete_step_response = response.json()

    print("Got successful complete steps response:")
    print(json.dumps(complete_step_response, indent=4))

#### Call `CompleteSteps` endpoint to complete multiple steps of the plan

In [ ]:
complete_steps(on_demand_trip_id, on_demand_step_ids)

## Step 6: Complete pre-schedule trip (trip-1)
Complete all the steps of trip-1 from drive to pick up location to drop off rider.

In [ ]:
complete_steps(pre_scheduled_trip_id, pre_scheduled_step_ids)

## Step 7: Clean up fleet

Remove the vehicle and fleet

In [ ]:
REMOVE_VEHICLE_URL = "https://api.rideos.ai/ride-hail-driver/v1/RemoveVehicle"

remove_vehicle_request = {
    "id": vehicle_id
}

response = requests.post(
    REMOVE_VEHICLE_URL,
    headers=AUTHORIZATION_HEADER,
    json=remove_vehicle_request
)
response.raise_for_status()

remove_vehicle_response = response.json()

print("Got successful remove vehicle response:")
print(json.dumps(remove_vehicle_response, indent=4))

### Remove the fleet

In [ ]:
REMOVE_FLEET_URL = "https://api.rideos.ai/ride-hail-operations/v1/RemoveFleet"

remove_fleet_request = {
    "id": fleet_id
}

response = requests.post(
    REMOVE_FLEET_URL,
    headers=AUTHORIZATION_HEADER,
    json=remove_fleet_request
)
response.raise_for_status()

remove_fleet_response = response.json()

print("Got successful remove fleet response:")
print(json.dumps(remove_fleet_response, indent=4))